In [1]:
# -*- coding: utf-8 -*-
"""
Created on Fri Feb 21 11:33:00 2025

@author: nelso
"""

import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
from scipy.special import comb
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import webbrowser

#Function to calculate the probability to have at least ka A and exactly kb B
def prob_at_least_ka_and_exactly_kb(KA, KB, total_cards, hand_size, ka, kb):
    total_hands = comb(total_cards, hand_size)
    Z = np.zeros((len(KA), len(KB)), dtype=float)
    
    for i, ka_val in enumerate(KA):
        for j, kb_val in enumerate(KB):
            prob_sum = 0
            other_cards = total_cards - ka_val - kb_val
            for xa in range(ka, min(ka_val, hand_size) + 1):
                prob_sum += (comb(ka_val, xa) * comb(other_cards, hand_size - xa - kb)) / comb(total_cards - kb_val, hand_size - kb)
            prob_exact_kbB = comb(kb_val, kb) * comb(total_cards - kb_val, hand_size - kb) / total_hands
            Z[i, j] = prob_sum * prob_exact_kbB
    
    return Z

#We use dash
app = dash.Dash(__name__)

#Layout of the application
app.layout = html.Div([
    html.H1("Distribution of Probabilities - At least ka and exactly kb (3D)"),
    dcc.Graph(id='3d-plot', style={'height': '700px'}),  #3D Plot
    html.Label("Total Cards"),
    dcc.Slider(id='total-cards', min=40, max=60, step=1, value=40, marks={i: str(i) for i in range(40, 61, 5)}),
    html.Label("Hand Size"),
    dcc.Slider(id='hand-size', min=1, max=60, step=1, value=5, marks={i: str(i) for i in range(1, 61, 5)}),
    html.Label("ka"),
    dcc.Slider(id='ka', min=0, max=10, step=1, value=2, marks={i: str(i) for i in range(0, 11)}),
    html.Label("kb"),
    dcc.Slider(id='kb', min=0, max=10, step=1, value=1, marks={i: str(i) for i in range(0, 11)})
])

# Callback to update the graph
@app.callback(
    Output('3d-plot', 'figure'),
    [Input('total-cards', 'value'),
     Input('hand-size', 'value'),
     Input('ka', 'value'),
     Input('kb', 'value')]
)
def update_plot(total_cards, hand_size, ka, kb):
    if hand_size > total_cards:
        hand_size = total_cards  #to be sure that hand-size is not higher than total_cards
    
    KA_values = np.arange(0, total_cards + 1)
    KB_values = np.arange(0, total_cards + 1)
    Z = prob_at_least_ka_and_exactly_kb(KA_values, KB_values, total_cards, hand_size, ka, kb)
    
    KA_grid, KB_grid = np.meshgrid(KA_values, KB_values)
    
    fig = go.Figure(data=[go.Surface(x=KA_grid, y=KB_grid, z=Z.T)])
    fig.update_layout(
        scene=dict(
            xaxis_title='Number of copies of A (KA)',
            yaxis_title='Number of copies of B (KB)',
            zaxis_title=f'Probability of drawing at least {ka} A and exactly {kb} B'
        ),
        title=f'Evolution of P(at least {ka} A and exactly {kb} B) depending on KA and KB',
        width=1000,
        height=700
    )
    return fig

#Open page automatically in browser
if __name__ == '__main__':
    webbrowser.open('http://127.0.0.1:8050')
    #app.run_server(debug=True)
    app.run(debug=True, port=8050)

C:\Users\nelso\AppData\Local\Temp\ipykernel_10708\846061322.py:27: RuntimeWarning:

invalid value encountered in scalar divide

